In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/Emotion Recogition/code/python_files

/content/drive/My Drive/Emotion Recogition/code/python_files


In [3]:
import numpy as np
import os
import sys
import pandas as pd

import wave
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

import keras
from keras.models import Sequential, Model
from sklearn.preprocessing import scale
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten,Dropout,GlobalAveragePooling2D,MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from scipy import signal
import matplotlib.pyplot as plot
from helper import *
from sklearn.preprocessing import LabelEncoder
from Layers import *
from Capsnet import CapsNet 
from keras import layers,losses,models,optimizers
from keras import backend as K


Using TensorFlow backend.


In [0]:
from helper import *

In [0]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [0]:
import pickle
with open(code_path + '/../data/'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [7]:
!pip install python_speech_features  

In [0]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav

In [9]:

X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used:
        fbank_feat = logfbank(ses_mod['signal'],nfilt=40)
        Sxx = np.transpose(fbank_feat)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        Sxx = scale(Sxx)
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

625
2034
(2034, 40, 300)
(625, 40, 300)


In [0]:
y  = pd.get_dummies(Y_train+Y_test)
y_train = y[0:len(Y_train)]
y_test = y[len(Y_train):]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
X_train = X_train.reshape(-1,40,300,1)
X_test = X_test.reshape(-1,40,300,1)

In [0]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
import matplotlib.pyplot as plt
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

In [0]:

def CapsNet(input_shape, n_class, routings):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=16*n_class))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    noise = layers.Input(shape=(n_class, 16))
    noised_digitcaps = layers.Add()([digitcaps, noise])
    masked_noised_y = Mask()([noised_digitcaps, y])
    manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model, eval_model, manipulate_model

In [0]:
def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

In [0]:
 model, eval_model, manipulate_model = CapsNet(input_shape=X_train.shape[1:],
                                                  n_class=4,
                                                  routings=3)

In [0]:
 # compile the model
    model.compile(optimizer=optimizers.Adam(lr=0.001,),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})


In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 40, 300, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 292, 256) 20992       input_4[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 12, 142, 256) 5308672     conv1[0][0]                      
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 54528, 8)     0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [0]:
 # callbacks 
   m_check = keras.callbacks.ModelCheckpoint(filepath = './capsulelog.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
   #lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * (0.9 ** 50))


In [0]:
Y_label = ( [np.where(r==1)[0][0] for r in y_train] )
early_stopping_monitor = EarlyStopping( monitor='val_loss',patience=20,verbose=1)
m_check = keras.callbacks.ModelCheckpoint(filepath = './capsule_encoder.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1 )

In [25]:
model.fit([X_train, y_train], [y_train, X_train], batch_size=50, epochs=50,
              validation_data=[[X_test, y_test], [y_test, X_test]], callbacks=[ m_check], shuffle= True,)

Train on 2034 samples, validate on 625 samples
Epoch 1/50


ResourceExhaustedError: ignored

In [0]:
model.save_weights(./result + '/trained_model.h5')
    print('Trained model saved to \'%s/trained_model.h5\'' % args.save_dir)

In [0]:
def train(model, data, args):
    """
    Training a CapsuleNet
    :param model: the CapsuleNet model
    :param data: a tuple containing training and testing data, like `((x_train, y_train), (x_test, y_test))`
    :param args: arguments
    :return: The trained model
    """
    # unpacking the data
    (x_train, y_train), (x_test, y_test) = data

    # callbacks
    log = callbacks.CSVLogger('./result'+ '/log.csv')
    tb = callbacks.TensorBoard(log_dir='./result'+ '/tensorboard-logs',
                               batch_size=args.batch_size, 
    checkpoint = callbacks.ModelCheckpoint('./result' + '/weights-{epoch:02d}.h5', monitor='val_capsnet_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
    lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * (0.9 ** 50))

    # compile the model
    model.compile(optimizer=optimizers.Adam(lr=0.001,),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})

    
    #Training without data augmentation:
    model.fit([x_train, y_train], [y_train, x_train], batch_size=args.batch_size, epochs=args.epochs,
              validation_data=[[x_test, y_test], [y_test, x_test]], callbacks=[log, tb, checkpoint, lr_decay])
   

    # Begin: Training with data augmentation ---------------------------------------------------------------------#
     """
    def train_generator(x, y, batch_size, shift_fraction=0.):
        train_datagen = ImageDataGenerator(width_shift_range=shift_fraction,
                                           height_shift_range=shift_fraction)  # shift up to 2 pixel for MNIST
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch], [y_batch, x_batch])
       

    # Training with data augmentation. If shift_fraction=0., also no augmentation.
    model.fit_generator(generator=train_generator(x_train, y_train, args.batch_size, args.shift_fraction),
                        steps_per_epoch=int(y_train.shape[0] / args.batch_size),
                        epochs=args.epochs,
                        validation_data=[[x_test, y_test], [y_test, x_test]],
                        callbacks=[log, tb, checkpoint, lr_decay])
    # End: Training with data augmentation -----------------------------------------------------------------------#
     """
    model.save_weights(args.save_dir + '/trained_model.h5')
    print('Trained model saved to \'%s/trained_model.h5\'' % args.save_dir)

    from utils import plot_log
    plot_log(args.save_dir + '/log.csv', show=True)

    return model

In [0]:
 model, eval_model, manipulate_model = CapsNet(input_shape=X_train.shape[1:],
                                                  n_class=len(np.unique(np.argmax(y_train, 1))),
                                                  routings=3)
    

IndentationError: ignored

In [0]:
if __name__ == "__main__":
    import os
    import argparse
    #from keras.preprocessing.image import ImageDataGenerator
    from keras import callbacks

    # setting the hyper parameters
    parser = argparse.ArgumentParser(description="Capsule Network on MNIST.")
    parser.add_argument('--epochs', default=50, type=int)
    parser.add_argument('--batch_size', default=100, type=int)
    parser.add_argument('--lr', default=0.001, type=float,
                        help="Initial learning rate")
    parser.add_argument('--lr_decay', default=0.9, type=float,
                        help="The value multiplied by lr at each epoch. Set a larger value for larger epochs")
    parser.add_argument('--lam_recon', default=0.392, type=float,
                        help="The coefficient for the loss of decoder")
    parser.add_argument('-r', '--routings', default=3, type=int,
                        help="Number of iterations used in routing algorithm. should > 0")
    parser.add_argument('--shift_fraction', default=0.1, type=float,
                        help="Fraction of pixels to shift at most in each direction.")
    parser.add_argument('--debug', action='store_true',
                        help="Save weights by TensorBoard")
    parser.add_argument('--save_dir', default='./result')
    parser.add_argument('-t', '--testing', action='store_true',
                        help="Test the trained model on testing dataset")
    parser.add_argument('--digit', default=5, type=int,
                        help="Digit to manipulate")
    parser.add_argument('-w', '--weights', default=None,
                        help="The path of the saved weights. Should be specified when testing")
    args = parser.parse_args()
    print(args)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load data
    #(x_train, y_train), (x_test, y_test) = load_mnist()

    # define model
    model, eval_model, manipulate_model = CapsNet(input_shape=X_train.shape[1:],
                                                  n_class=len(np.unique(np.argmax(y_train, 1))),
                                                  routings=3)
    model.summary()

    # train or test
    if args.weights is not None:  # init the model weights with provided one
        model.load_weights(args.weights)
    if not args.testing:
        train(model=model, data=((X_train, y_train), (X_test, y_test)), args=args)
    

usage: ipykernel_launcher.py [-h] [--epochs EPOCHS] [--batch_size BATCH_SIZE]
                             [--lr LR] [--lr_decay LR_DECAY]
                             [--lam_recon LAM_RECON] [-r ROUTINGS]
                             [--shift_fraction SHIFT_FRACTION] [--debug]
                             [--save_dir SAVE_DIR] [-t] [--digit DIGIT]
                             [-w WEIGHTS]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-dfee3e68-bda9-40d2-adb2-2cb2a1deabf0.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
def make_model():
    in_layer = Input(shape=(40, 200,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)  
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) # softmax output
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
m_check = keras.callbacks.ModelCheckpoint(filepath = './cnn_logfilter_default.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check] 
                )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 18s 9ms/step - loss: 2.0054 - acc: 0.3535 - val_loss: 2.9518 - val_acc: 0.2592

Epoch 00001: val_acc improved from -inf to 0.25920, saving model to ./cnn_logfilter_default.h5
Epoch 2/100
2034/2034 [==============================] - 10s 5ms/step - loss: 1.4955 - acc: 0.4169 - val_loss: 1.3380 - val_acc: 0.5408

Epoch 00002: val_acc improved from 0.25920 to 0.54080, saving model to ./cnn_logfilter_default.h5
Epoch 3/100
2034/2034 [==============================] - 10s 5ms/step - loss: 1.4156 - acc: 0.4326 - val_loss: 1.0730 - val_acc: 0.5504

Epoch 00003: val_acc improved from 0.54080 to 0.55040, saving model to ./cnn_logfilter_default.h5
Epoch 4/100
2034/2034 [==============================] - 10s 5ms/step - loss: 1.3081 - acc: 0.4454 - val_loss: 1.2375 - val_acc: 0.5584

Epoch 00004: val_acc improved from 0.55040 to 0.55840, saving model to ./cnn_logfilter_default.h5
Epoch 5/100
2034

In [0]:
best_model = load_model('./cnn_logfilter_default.h5')

In [0]:
y_pred = best_model.predict(X_test)

In [0]:
y_pred

array([[0.00766669, 0.0169646 , 0.06303743, 0.9123313 ],
       [0.10299007, 0.17346919, 0.47585446, 0.24768631],
       [0.22285958, 0.62268   , 0.1067394 , 0.04772101],
       ...,
       [0.25580174, 0.35741118, 0.37001726, 0.01676979],
       [0.14690156, 0.24690948, 0.42092654, 0.18526238],
       [0.23115315, 0.3139801 , 0.4261075 , 0.0287592 ]], dtype=float32)

In [0]:
from sklearn.metrics import classification_report, accuracy_score

In [0]:
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])) :
    if y_pred[i][j]==max(y_pred[i]) :
      y_pred[i][j] = 1
    else:
      y_pred[i][j]=0

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.65      0.47      0.54       174
           2       0.62      0.78      0.69       287
           3       0.70      0.74      0.72       133

   micro avg       0.65      0.65      0.65       625
   macro avg       0.49      0.50      0.49       625
weighted avg       0.62      0.65      0.62       625
 samples avg       0.65      0.65      0.65       625

